# The Manual Burden vs The Pydantic Way

In [ ]:
# data = item_id, quantity
# user_dict = user_id, email_id

In [ ]:
def process_order(data: dict):
    if "item_id" not in data or "quantity" not in data:
        raise ValueError("Missing required fields!")
    if not isinstance(data["item_id"], int):
        raise TypeError("item_id must be an integer!")
    if data["quantity"] <= 0:
        raise ValueError("Quantity must be positive")

data = {"item_id": "101", "quantity": "5"}
process_order(data)

In [ ]:
from pydantic import BaseModel, ValidationError, PositiveInt

class Order(BaseModel):
    item_id: int
    quantity: PositiveInt

def process_order(data: dict):
    try:
        order = Order(**data)
        print(f"Processing order for item {order.item_id} with quantity {order.quantity}")
    except ValidationError as e:
        print("Alert! Invalid data recieved!")
        print(e.errors())
data = {"item_id": "101", "quantity": "5"}
process_order(data)

# Dataclass vs Pydantic

In [ ]:
from dataclasses import dataclass
from pydantic import BaseModel, ValidationError

In [ ]:
@dataclass
class UserDC:
    user_id: int

class UserPY(BaseModel):
    user_id: int

In [ ]:
raw_data = {"user_id": "abc"}
user_dc = UserDC(**raw_data)
print(f"Dataclass value: {user_dc.user_id}")

In [ ]:
try:
    user_py = UserPY(**raw_data)
except ValidationError as e:
    print("Pydantic caught the error!")

# The Internal Logic

In [ ]:
# Phase 1: Definition Time. When Python first reads your script and sees class User(BaseModel), Pydantic’s Metaclass kicks in.
# It inspects your type hints, looks at your default values, and builds a 'Validation Schema.'
# In Pydantic V2, it then hands this schema over to the Rust core, which compiles a highly optimized validator specifically for that class.

# Phase 2: Runtime. This is when you actually pass data into the model.
# Because the validator was already 'compiled' at start-up, the actual checking of your data happens at near-C speeds."

## The Validation Lifecycle

In [ ]:
# Raw Input Intake: It takes your dictionary or JSON.

# Key Mapping: It checks for aliases. If your API sends ID but your model uses user_id, Pydantic maps them now.

# The Recursive Dive: If you have nested models, Pydantic dives into the deepest level first.
# It validates the 'child' before it validates the 'parent.'

# Coercion & Cleaning: It attempts to fix types.

# The Final Assembly: Only if every single check passes does the __dict__ of the object get populated.

In [ ]:
from pydantic import BaseModel, PositiveInt
from typing import List

In [ ]:
class Item(BaseModel):
    name: str
    count: PositiveInt


class Category(BaseModel):
    title: str
    items: List[Item]

In [ ]:
data = {
    "title": "Electronics",
    "items": [
        {"name": "Laptop",  "count": 5},
        {"name": "Mouse", "count": 10}
    ]
    
}

cat = Category(**data)
print(f"Validated {cat.title} with {len(cat.items)} items.")

In [ ]:
## Error Batching for Performance

In [ ]:
class UserProfile(BaseModel):
    username: str
    age: PositiveInt

invalid_date = {"username": 123, "age": -5}

In [ ]:
try:
    UserProfile(**invalid_date)
except ValidationError as e:
    # .erorrs() <- collection of errors if any
    for error in e.errors():
        print(error)

# Required, Optional and Default Fields

In [ ]:
from pydantic import BaseModel, Field

In [ ]:
class Employee(BaseModel):
    emp_id: int #Simple Required Field
    name: str = Field(..., min_length=2, decription="Employee Full Name")
emp = Employee(emp_id=101, name = "A")

In [ ]:
class User(BaseModel):
    username: str
    is_active: bool = True #Default value
    role: str = "Subscriber"

u = User(username="Abhishek")
print(u.is_active)
print(u.role)

In [ ]:
# default_factory

In [ ]:
from datetime import datetime
from pydantic import BaseModel, Field
from uuid import uuid4

In [ ]:
class Log(BaseModel):
    log_id: str = Field(default_factory=lambda :str(uuid4()))
    timestamp: datetime = Field(default_factory=datetime.now)

l1 = Log()
l2 = Log()

print(f"Log 1 timestamp: {l1.timestamp}")
print(f"Log 2 timestamp: {l2.timestamp}")

In [ ]:
## Optional Fields

In [ ]:
from typing import Optional

In [ ]:
class Profile(BaseModel):
    bio: str | None = None
    website: Optional[str] = None

In [ ]:
p = Profile()
print(p.bio)

# Annotated

In [ ]:
from typing import Annotated
from pydantic import BaseModel, Field

In [ ]:
RequiredName = Annotated[str, Field(..., min_length = 3, max_length=50)]

class Admin(BaseModel):
    name: RequiredName
    department: str

class Customer(BaseModel):
    name: RequiredName
    email: str

In [ ]:
RequiredName = Annotated[str, Field(default="admin")]

class Admin(BaseModel):
    name: RequiredName
    department: str

class Customer(BaseModel):
    name: RequiredName
    email: str

In [ ]:
RequiredName = Annotated[str | None, Field(default=None, max_length=150)]

class Admin(BaseModel):
    name: RequiredName
    department: str

class Customer(BaseModel):
    name: RequiredName
    email: str

# Field Validator

In [1]:
# "Age": int
# "Age": int (>18 & <120)

In [2]:
# mode = "before"
# Price = "$5, 000"

In [3]:
from pydantic import BaseModel, field_validator
import re

In [4]:
class Product(BaseModel):
    name: str
    price: float

    @field_validator("price", mode="before")
    @classmethod
    def clean_currency(cls, v: any):
        if isinstance(v, str):
            v = re.sub(r'[^\d.]', '', v)
        return v

item = Product(name="MacBook", price="$ 2,499")
print(item.price)

2499.0


In [7]:
class UserProfile(BaseModel):
    username: str
    age: int

    @field_validator("age", mode="after")
    @classmethod
    def age_limit(cls, v: any):
        if v < 18:
            raise ValueError("Age can't be less than 18")
        return v

user = UserProfile(username="Akaash", age="25")
print(user.age)

25


In [8]:
## Annotated appoach

In [9]:
from typing import Annotated
from pydantic import AfterValidator, Field

In [11]:

def ensure_no_space(v: str):
    if " " in v:
        raise ValueError("Spaces not allowed!")
    return v.lower()

user_name = Annotated[str, Field(min_length=3), AfterValidator(ensure_no_space)]

class Admin(BaseModel):
    user_id: user_name


class Student(BaseModel):
    profile_name: user_name

admin = Admin(user_id="Abhishek")
print(admin.user_id)

abhishek


In [12]:
from pydantic import BeforeValidator

In [14]:
# "rs. 500"
def clean_currency(v:any):
    if isinstance(v, str):
        return v.replace("rs. ", "").replace(",", "")
    return v



cleanint = Annotated[int, BeforeValidator(clean_currency)]

class Product(BaseModel):
    price: cleanint

p = Product(price="rs. 1,500")
print(p.price)

1500


# Model Validators

In [15]:
#passward and confirm passward

In [17]:
from pydantic import BaseModel, model_validator

In [20]:
class UserSignUp(BaseModel):
    password: str
    confirm_password: str

    @model_validator(mode="after")
    def check_passwords_match(self):
        if self.password != self.confirm_password:
            raise ValueError("Passwords don't match!")
        return self


try:
    user = UserSignUp(password="BrainByte123", confirm_password="WrongPassword")
except ValueError as e:
    print(e)

1 validation error for UserSignUp
  Value error, Passwords don't match! [type=value_error, input_value={'password': 'BrainByte12...sword': 'WrongPassword'}, input_type=dict]
    For further information visit https://errors.pydantic.dev/2.8/v/value_error


In [23]:
from datetime import date

In [26]:
class TravelBooking(BaseModel):
    destination: str
    start_date: date
    end_date: date

    @model_validator(mode="after")
    def validate_dates(self):
        if self.end_date <= self.start_date:
            raise ValueError("End date should be after start date!")

        return self

booking = TravelBooking(
    destination="Goa", 
    start_date=date(2026, 5, 20), 
    end_date=date(2026, 5, 21)
)

print(booking.destination)

Goa


In [27]:
# name, is_premium, membership_id 

In [28]:
from typing import Optional

In [30]:
class Member(BaseModel):
    name: str
    is_premium: bool
    membership_id: Optional[str] = None

    @model_validator(mode="after")
    def check_premium_id(self):
        if self.is_premium and not self.membership_id:
            raise ValueError("Premium account should have membership id")

        return self


m2 = Member(name="Suresh", is_premium=False)

In [34]:
class Product(BaseModel):
    name: str
    price: float

    @model_validator(mode="before")
    def clean_currency(cls, data):
        raw = data.get("price")
        if isinstance(raw, str):
            raw = raw.replace("$", "").replace(",", "")
            data["price"] = raw

        return data

p = Product(name="laptop", price="$1,200.50")
print(p)

name='laptop' price=1200.5


In [38]:
# UserSignup(pass="abc", confirm="abc")

class UserSignup(BaseModel):
    password: str
    confirm_password: str


    @model_validator(mode="before")

    def unify_password_keys(cls, data):
        if "pass_" in data:
            data["password"] = data.pop("pass_")
        if "confirm" in data:
            data["confirm_password"] = data.pop("confirm") 
        return data

user = UserSignup(pass_="abc", confirm="abc")
print(user)

password='abc' confirm_password='abc'


# Cumputed Fields

In [1]:
# @cumputed_field

In [6]:
from pydantic import BaseModel, computed_field

In [7]:
class Subcriber(BaseModel):
    first_name: str
    last_name: str

    @computed_field
    @property
    def full_name(self) -> str:
        return f"{self.first_name} {self.last_name}"


sub = Subcriber(first_name="Abhishek", last_name="kumar")
print(sub.model_dump())

{'first_name': 'Abhishek', 'last_name': 'kumar', 'full_name': 'Abhishek kumar'}


In [9]:
class VideoStats(BaseModel):
    video_title: str
    views: int
    likes: int

    @computed_field
    @property
    def engagement_rate(self)-> str:
        if self.views == 0:
            return "0%"
        rate = (self.likes / self.views)*100
        return f"{round(rate, 2)}%"

    @computed_field
    @property
    def is_viral(self)-> bool:
        return self.views > 100000

stats = VideoStats(video_title="Pydantic Tutorial", views=150000, likes=12000)
print(stats.model_dump_json())

{"video_title":"Pydantic Tutorial","views":150000,"likes":12000,"engagement_rate":"8.0%","is_viral":true}


# Nested Models

In [1]:
from pydantic import BaseModel

In [2]:
class Address(BaseModel):
    city: str
    zip_code: str

class User(BaseModel):
    username: str
    email: str
    address: Address

In [4]:
data = {
    "username": "Abhishek",
    "email": "abhishek111@gmail.com",
    "address":
        {
            "city": "Pune",
            "zip_code": "411014"
}
}
user = User(**data)
print(user.address.city)

Pune


In [5]:
# List[Model]

In [6]:
from typing import List
from pydantic import BaseModel, PositiveInt

In [7]:
class OrderItems(BaseModel):
    product_name: str
    quantity: PositiveInt

class Order(BaseModel):
    order_id: int
    items: List[OrderItems]  #List of nested models

In [11]:
order_data = {
    "order_id": 101,
    "items": [
        {"product_name": "Keyboard", "quantity": 6},
        {"product_name": "Mouse", "quantity": 12}
    ]
}
order = Order(**order_data)
print(len(order.items))
print(order)

2
order_id=101 items=[OrderItems(product_name='Keyboard', quantity=6), OrderItems(product_name='Mouse', quantity=12)]


# Serialization & Dumping

In [15]:
# model_dump = Python Dict
# model_dump_json = JSON string

In [16]:
from pydantic import BaseModel, Field

In [20]:
class User(BaseModel):
    username: str
    email: str
    password: str = Field(exclude=True)


In [21]:
user = User(username="Abhishek", email="xyz@gmail.com", password="123456789")
print(user.model_dump())

{'username': 'Abhishek', 'email': 'xyz@gmail.com'}


In [22]:
# User-ID, FirstName

In [27]:
class ExternameAPIUser(BaseModel):
    user_id: int = Field(alias="User-ID")
    full_name: str = Field(alias="Full-Name")

In [31]:
data = {"User-ID": 101, "Full-Name": "Abhishek"}

user = ExternameAPIUser(**data)
print(user.model_dump(by_alias=True))

{'User-ID': 101, 'Full-Name': 'Abhishek'}


In [32]:
# exclude_none = removes all none values
# exclude_unset = removes the default values

In [33]:
class ProfileUpdate(BaseModel):
    bio: str | None = None
    website: str | None = None
    location: str = "India"

In [34]:
user = ProfileUpdate(bio="Learning Pydantic")
print(update.model_dump())

{'bio': 'Learning Pydantic!', 'website': None, 'location': 'India'}


In [35]:
print(update.model_dump(exclude_none=True))

{'bio': 'Learning Pydantic!', 'location': 'India'}


In [ ]:
print(update.model_dump(exclude_none=True))